In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

max_iters = 10000
learning_rate = 3e-3
eval_iters = 250
n_embd = 384

dropout = 0.2

cpu


In [2]:
# read in our corpus into text string

with open('data\philosophers.txt','r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])



=== PLATO ===

How you, O Athenians, have been affected by my accusers, I cannot tell;
but I know that they almost made me forget who I was—so persuasively
did they speak; and yet they have hardly 


In [5]:
# encoder and decoder for strings to integers and vice versa

chars = sorted(set(text))
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
# use first 80% of corpus for training; remaining 20% for validation
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocabulary_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

    def forward(self, index, targets=None):
        print(f'index: {index}')
        logits = self.token_embeddings_table(index)  # (B,T,C)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        if targets is None:
            loss = None
        else:
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits.view(B, T, C), loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)          # (B,T,C)
            #print(f'index {index}')
            print(logits)
            logits = logits[:, -1, :]                # take last time step -> (B,C)
            #print('last')
            #print(logits)
            probs = F.softmax(logits, dim=-1)        # turn into probabilities
            #print('probs')
            #print(probs)
            index_next = torch.multinomial(probs, 1) # sample next token -> (B,1)
            #print(index_next)
            index = torch.cat((index, index_next), dim=1)
        return index
